# Credit Risk Cleanup

Run this notebook if you are done experimenting with this demo, or if you wish to start again with a clean slate.

**RUNNING THE FOLLOWING CODE WILL REMOVE FILES AND PROCESSES CREATED BY THE PREVIOUS EXAMPLE NOTEBOOKS.**

The notebook progresses in reverse order of how the files and processes were added. (The reverse order makes it possible to partially revert changes by running cells up to a certain point.)

#### Setup

In [1]:
# Imports
import os
import time
import psutil

#### Remove Trained Model File

This removes the model that was created and saved in [03_Credit_Risk_Model_Training.ipynb](03_Credit_Risk_Model_Training.ipynb).

In [2]:
# Remove the model file that was saved in model training.
model_path = "./rf_model.pkl"
os.remove(model_path)

#### Shutdown Servers

Shut down the servers that were launched in [02_Deploying_the_Feature_Store.ipynb](02_Deploying_the_Feature_Store.ipynb); also remove the `server_proc.txt` that held the process PIDs.

In [3]:
# Load server process objects
server_pids = open("server_proc.txt").readlines()
offline_server_proc = psutil.Process(int(server_pids[0].strip()))
online_server_proc = psutil.Process(int(server_pids[1].strip()))

In [4]:
# Verify if servers are running
def verify_servers():
    # online server
    print(f"Online server :            {online_server_proc}")
    print(f"Online server is running:  {online_server_proc.is_running()}", end='\n\n')
    # offline server
    print(f"Offline server PID:        {offline_server_proc}")
    print(f"Offline server is running: {offline_server_proc.is_running()}")
    
verify_servers()

Online server :            psutil.Process(pid=44621, name='python3.11', status='running', started='14:19:05')
Online server is running:  True

Offline server PID:        psutil.Process(pid=44594, name='python3.11', status='running', started='14:19:03')
Offline server is running: True


In [5]:
# Terminate offline server
offline_server_proc.terminate()

In [6]:
# Terminate online server (master and worker)
for child in online_server_proc.children(recursive=True):
    child.terminate()
online_server_proc.terminate()
time.sleep(2)

In [7]:
# Verify termination
verify_servers()

Online server :            psutil.Process(pid=44621, name='python3.11', status='terminated', started='14:19:05')
Online server is running:  False

Offline server PID:        psutil.Process(pid=44594, name='python3.11', status='terminated', started='14:19:03')
Offline server is running: False


In [8]:
# Remove server_proc.txt (file for keeping track of pids)
os.remove("server_proc.txt")

#### Remove Feast Applied Configuration Files

Remove the registry and online store (SQLite) files created on`feast apply` created in [02_Deploying_the_Feature_Store.ipynb](02_Deploying_the_Feature_Store.ipynb).

In [9]:
os.remove("Feature_Store/data/online_store.db")
os.remove("Feature_Store/data/registry.db")

#### Remove Feast Configuration Files

Remove the configution and feature definition files created in [02_Deploying_the_Feature_Store.ipynb](02_Deploying_the_Feature_Store.ipynb).

In [10]:
os.remove("Feature_Store/feature_store.yaml")
os.remove("Feature_Store/feature_definitions.py")

#### Remove Data Files

Remove the data files created in [01_Credit_Risk_Data_Prep.ipynb](01_Credit_Risk_Data_Prep.ipynb).

In [11]:
for f in ["data_a.parquet", "data_b.parquet", "labels.parquet"]:
    os.remove(f"Feature_Store/data/{f}")
os.rmdir("Feature_Store/data")
os.rmdir("Feature_Store")